# 可视化模型的 Scale&Shift 参数
## ViT-b16

In [3]:
%cd ..

/home/zxy/SSF


In [4]:
import sys
import os
PROJECT_ROOT="/home/wyh/PEFT/SSF"
PROJECT_ROOT="/home/wyh/SSF"
SAVED_ROOT="/media/disk1/wyh/SSF"
sys.path.append(PROJECT_ROOT)
import torch
import timm
from timm.models import create_model, safe_model_name, resume_checkpoint, load_checkpoint,\
    convert_splitbn_model, model_parameters
from models import vision_transformer, swin_transformer, convnext, as_mlp



In [32]:
MODEL_NAME="vit_base_patch16_224_in21k"
MODEL_PATH="vit_base_patch16_224_in21k/cifar_100/ssf/20230521-144333-vit_base_patch16_224_in21k-224/model_best.pth.tar"
# 235
MODEL_PATH="vit_base_patch16_224_in21k/cifar_100/ssf/20230523-171946-vit_base_patch16_224_in21k-224/model_best.pth.tar"
MODEL_PATH="vit_base_patch16_224_in21k/cifar_100/pruning/20230608-204025-vit_base_patch16_224_in21k-224/model_best.pth.tar"
MODEL_PATH=os.path.join(SAVED_ROOT,MODEL_PATH)
MODEL_PATH="/media/disk1/wyh/SSF/vit_base_patch16_224_in21k/cifar_100/pruning_no_ema/20230608-234152-vit_base_patch16_224_in21k-224/checkpoint-90.pth.tar"
MODEL_PATH="/media/disk1/wyh/SSF/vit_base_patch16_224_in21k/cifar_100/pruning_no_ema/20230608-234152-vit_base_patch16_224_in21k-224/model_best.pth.tar"
MODEL_PATH="/media/disk1/wyh/SSF/vit_base_patch16_224_in21k/cifar_100/pruning_no_ema/20230608-234424-vit_base_patch16_224_in21k-224/model_best.pth.tar"
MODEL_PATH='/data/hjy/SSF/vit_base_patch16_224_in21k/vtab/pets/pets_2.73_221_0.1_0.01.pth.tar'
# MODEL_PATH='/data/hjy/SSF/vit_base_patch16_224_in21k/vtab/pets/pruning_retrain_full/20230812-211342-vit_base_patch16_224_in21k-224/checkpoint-280.pth.tar'
METHOD_NAME="ssf"
NUM_CLASSES=37

In [33]:

model = create_model(
    MODEL_NAME,
    pretrained=True,
    num_classes=NUM_CLASSES,
    drop_rate=0.0,
    drop_path_rate=0.0,
    drop_block_rate=None,
    global_pool=None,
    bn_momentum=None,
    bn_eps=None,
    scriptable=False,
    checkpoint_path='',
    tuning_mode=METHOD_NAME)
resume_epoch = resume_checkpoint(
    model, MODEL_PATH,
    optimizer=None,
    loss_scaler=None,
    log_info=True)


In [4]:
# for i,(name,module) in enumerate(model.named_modules()) :
#     # print(name)
#     cw=False
#     for m in module.children():
#         if hasattr(m,"weight"):
#             cw=True
#     if cw:
#     # if hasattr(module,"weight"):
#         print(i,"\t",name)
#         print("PARAMETER: ")
#         for n,p in module.named_parameters():
#             print(n,"\t",p.shape)
#         print("DICT:")
#         # print(type(module.weight),module.weight.size())
#         for k in module.__dict__:
#             print("\t",k)
# for j,(name,parameters) in enumerate(model.named_parameters()):
#     key=["ssf_scale","ssf_scale"]
#     if any([k in name for k in key]):
#         print("{}\t{}\t{}\t{}".format(j,name,parameters.shape,parameters.data.shape))        
    

In [35]:
ssf_scale,ssf_shift={},{}
for name, para in model.named_parameters():
    if "ssf_scale" in name:
        ssf_scale[name]=para.data.detach().cpu().numpy()
    elif "ssf_shift" in name:
        ssf_shift[name]=para.data.detach().cpu().numpy()
    else:
        pass
print("MAX Scale: ", max([s.max() for s in ssf_scale_list]))
print("MIN Scale: ", min([s.min() for s in ssf_scale_list]))
print("AVE Scale: ", sum([sum(s) for s in ssf_scale_list])/sum([len(s) for s in ssf_scale_list]))
print("MAX Shift: ", max([s.max() for s in ssf_shift_list]))
print("MIN Shift: ", min([s.min() for s in ssf_shift_list]))
print("AVE Shift: ", sum([sum(s) for s in ssf_shift_list])/sum([len(s) for s in ssf_shift_list]))

MAX Scale:  3.541211
MIN Scale:  -2.802487
AVE Scale:  0.04294330928610473
MAX Shift:  2.8979683
MIN Shift:  -3.0752964
AVE Shift:  -0.0042768097495824295


In [ ]:
# show basic info of ssf_scale and ssf_shift:
print("Length: ",len(ssf_scale),"\t",len(ssf_shift))
print("SSF_SCALE:")
for k in ssf_scale:
    print(k,ssf_scale[k].shape)
print("SSF_SHIFT:")
for k in ssf_shift:
    print(k,ssf_shift[k].shape)

In [31]:
import warnings
warnings. filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision.models as models

def draw6(row1,name=None):
    data=row1
    arr = data
    up_mean=np.mean(np.array([a.max() for a in arr]))
    lw_mean=np.mean(np.array([a.min() for a in arr]))
    num_rows = len(arr)
    fig, axs = plt.subplots(ncols=num_rows, nrows=1, sharey=True,gridspec_kw={"hspace": 0, "wspace": 0})
    fig.set_size_inches(20, 4)
    lw=lw_mean*1.5 if lw_mean<0 else lw_mean*0.5
    ran=(lw,up_mean*1.5)
    ran=(lw,up_mean*1.5)
    for i in range(num_rows):
        # sns.histplot(y=arr[i], ax=axs[i], kde=True, stat="density")
        # sns.distplot(arr[i], ax=axs[i], kde=True, stat="density",hist_kws={'range': (0.0, 0.2)})
        color="blue"
        if arr[i].max() > up_mean*1.2 or arr[i].min() < lw_mean*0.8:
            color="red"
        sns.distplot(arr[i], ax=axs[i], kde=True, vertical=True,color=color)
        # sns.distplot(arr[i], ax=axs[i], kde=True, vertical=True,color=color,\
        #     hist_kws={'range': ran},kde_kws={'clip': ran})
        # sns.histplot(data, clip=(0, 10))
        axs[i].set_xticks([])
        # axs[i].set_xlabel(f"{i+1}")
        label=""
        if 'attn' in name[i]:
            label+="a"
        elif 'mlp' in name[i]:
            label+="m"
        else:
            label+="b"
        label+=name[i].split("_")[-1]
        axs[i].set_xlabel(label)
    plt.tight_layout()
    # sns.set_style('white')
    sns.set(style='whitegrid')
    sns.despine(right=True,left=True)
    plt.show()
ssf_scale_list,ssf_shift_list,ssf_scale_name,ssf_shift_name=[],[],[],[]
for name, para in model.named_parameters():
    if "ssf_scale" in name:
        ssf_scale_list.append(para.data.detach().cpu().numpy())
        ssf_scale_name.append(name)
    elif "ssf_shift" in name:
        ssf_shift_list.append(para.data.detach().cpu().numpy())
        ssf_shift_name.append(name)
    else:
        pass
# draw6(ssf_scale_list,ssf_scale_name)
# draw6(ssf_shift_list,ssf_shift_name)
print("MAX Scale: ", max([s.max() for s in ssf_scale_list]))
print("MIN Scale: ", min([s.min() for s in ssf_scale_list]))
print("AVE Scale: ", sum([sum(s) for s in ssf_scale_list])/sum([len(s) for s in ssf_scale_list]))
print("MAX Shift: ", max([s.max() for s in ssf_shift_list]))
print("MIN Shift: ", min([s.min() for s in ssf_shift_list]))
print("AVE Shift: ", sum([sum(s) for s in ssf_shift_list])/sum([len(s) for s in ssf_shift_list]))

MAX Scale:  3.541211
MIN Scale:  -2.802487
AVE Scale:  0.04294330928610473
MAX Shift:  2.8979683
MIN Shift:  -3.0752964
AVE Shift:  -0.0042768097495824295


In [25]:
# print("MIN Scale: ", min([s.min() for s in ssf_scale_list]))
# print("MAX Shift: ", max([s.max() for s in ssf_shift_list]))
sc_num=0
sc_pnm=0
for i,(sc,sh) in enumerate(zip(ssf_scale_list,ssf_shift_list)):
    # select indexes from sc, where abs value is less than 0.02
    # indexes=np.where(np.abs(sc)<0.02)
    indexes=np.argwhere(np.abs(sc)<0.01)
    subsc=np.take(sc,indexes)
    sc_pnm+=len(subsc)
    sc_num+=len(sc)
    subsh=np.take(sh,indexes)
    
    # subsh=np.take(sh,indexes)
    print("{:.2f} Layer {}:\t{}/{}\t{:.2f}%".format(np.abs(subsh).mean(), i,len(subsc),len(sc),len(subsc)/len(sc)*100))
print("Total:\t{}/{}\t{:.2f}%".format(sc_pnm,sc_num,sc_pnm/sc_num*100))

0.00 Layer 0:	369/768	48.05%
0.06 Layer 1:	181/768	23.57%
0.09 Layer 2:	324/768	42.19%
0.17 Layer 3:	226/768	29.43%
0.09 Layer 4:	882/2304	38.28%
0.02 Layer 5:	190/768	24.74%
0.08 Layer 6:	1106/3072	36.00%
0.02 Layer 7:	187/768	24.35%
0.16 Layer 8:	312/768	40.62%
0.21 Layer 9:	265/768	34.51%
0.13 Layer 10:	935/2304	40.58%
0.02 Layer 11:	194/768	25.26%
0.03 Layer 12:	1462/3072	47.59%
0.02 Layer 13:	276/768	35.94%
0.06 Layer 14:	310/768	40.36%
0.13 Layer 15:	294/768	38.28%
0.06 Layer 16:	1000/2304	43.40%
0.01 Layer 17:	281/768	36.59%
0.03 Layer 18:	1425/3072	46.39%
0.02 Layer 19:	317/768	41.28%
0.05 Layer 20:	327/768	42.58%
0.06 Layer 21:	327/768	42.58%
0.05 Layer 22:	966/2304	41.93%
0.02 Layer 23:	291/768	37.89%
0.03 Layer 24:	1327/3072	43.20%
0.02 Layer 25:	337/768	43.88%
0.05 Layer 26:	330/768	42.97%
0.07 Layer 27:	328/768	42.71%
0.02 Layer 28:	989/2304	42.93%
0.02 Layer 29:	281/768	36.59%
0.03 Layer 30:	1310/3072	42.64%
0.02 Layer 31:	362/768	47.14%
0.05 Layer 32:	304/768	39.58%
0.04

In [9]:
# def print_outler(row1,name=None):
#     data=row1
#     arr = data
#     up_mean=np.mean(np.array([a.max() for a in arr]))
#     lw_mean=np.mean(np.array([a.min() for a in arr]))
#     num_rows = len(arr)
#     lw=lw_mean*1.5 if lw_mean<0 else lw_mean*0.5
#     ran=(lw,up_mean*1.5)
#     for i in range(len(row1)):
#         if arr[i].max() > up_mean*1.2 or arr[i].min() < lw_mean*0.8:
#             print(name[i])
# print_outler(ssf_scale_list,ssf_scale_name)
# print_outler(ssf_shift_list,ssf_shift_name)


# 运行计时
5.23 00:35 开始运行 ./train_scripts/vit/cifar_vtab